In [ ]:
from typing import Annotated,Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import RemoveMessage


In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Modules")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Modules


In [3]:


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
tool = TavilySearch(max_results=3)
tools = [tool]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


In [4]:
llm_with_tools.invoke('안녕하세요 저는 김태훈입니다')

AIMessage(content='안녕하세요, 김태훈님! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 98, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CIvSYO245gllwS9JUmsnCUKyRddxo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3df621dd-506b-4276-86a2-ec920ea3a0b9-0', usage_metadata={'input_tokens': 98, 'output_tokens': 16, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    result = llm_with_tools.invoke(state["messages"])
    return State({"messages": [result]})

# 메시지 개수가 3개 초과 시 오래된 메시지 삭제 및 최신 메시지만 유지
def delete_messages(state:State):
    messages = state["messages"]
    if len(messages) > 3:
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:-3]]}


# 메시지 상태에 따른 다음 실행 노드 결정 로직
def should_continue(state: State) -> Literal["tools", "delete_messages"]:
    """Return the next node to execute."""
    if messages := state.get("messages", []):
        # 가장 최근 AI 메시지 추출
        ai_message = messages[-1]
    else:
        # 입력 상태에 메시지가 없는 경우 예외 발생
        raise ValueError(f"No messages found in input state to tool_edge: {state}")

    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tools"
    # 도구 호출이 없는 경우 "END" 반환
    return 'delete_messages'


# 상태 그래프 생성
graph_state = StateGraph(State)

# 챗봇 노드 추가
graph_state.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[tool])

# 도구 노드 추가
graph_state.add_node("tools", tool_node)

graph_state.add_node("delete_messages", delete_messages)

# 조건부 엣지
graph_state.add_conditional_edges(
    "chatbot",
    should_continue,
)

########## 4. 엣지 추가 ##########

graph_state.add_edge(START, "chatbot")

graph_state.add_edge("tools", "chatbot")

graph_state.add_edge("chatbot", "delete_messages")

graph_state.add_edge("delete_messages", END)

In [15]:
# 메모리 저장소 생성
memory = MemorySaver()
graph = graph_state.compile(checkpointer=memory)
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot(chatbot)
	tools(tools)
	delete_messages(delete_messages)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot;
	chatbot --> delete_messages;
	chatbot -.-> tools;
	tools --> chatbot;
	delete_messages --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [7]:
from langchain_teddynote.graphs import visualize_graph
# 그래프 시각화
visualize_graph(graph)

그래프 시각화 실패 (추가 종속성 필요): Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`
ASCII로 그래프 표시:
          +-----------+               
          | __start__ |               
          +-----------+               
                *                     
                *                     
                *                     
          +---------+                 
          | chatbot |                 
          +---------+                 
         ***         ***              
        *               *             
      **                 **           
+-------+         +-----------------+ 
| tools |         | delete_messages | 
+---

In [16]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=6,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "0012"},  # 스레드 ID 설정
)

In [17]:
# LangChain 메시지 처리를 위한 HumanMessage 클래스 임포트
from langchain_core.messages import HumanMessage

# 1번째 질문 수행
# input_message = HumanMessage(
#     content="안녕하세요! 제 이름은 Teddy입니다. 잘 부탁드립니다."
# )

question="안녕하세요! 제 이름은 Teddy입니다. 잘 부탁드립니다."

for event in graph.stream({"messages": [question]}, config=config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕하세요! 제 이름은 Teddy입니다. 잘 부탁드립니다.')]
[('human', '안녕하세요! 제 이름은 Teddy입니다. 잘 부탁드립니다.'), ('ai', '안녕하세요, Teddy님! 만나서 반갑습니다. 어떻게 도와드릴까요?')]


In [18]:
question="내이름이머라고 ?."

for event in graph.stream({"messages": [question]}, config=config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕하세요! 제 이름은 Teddy입니다. 잘 부탁드립니다.'), ('ai', '안녕하세요, Teddy님! 만나서 반갑습니다. 어떻게 도와드릴까요?'), ('human', '내이름이머라고 ?.')]
[('human', '안녕하세요! 제 이름은 Teddy입니다. 잘 부탁드립니다.'), ('ai', '안녕하세요, Teddy님! 만나서 반갑습니다. 어떻게 도와드릴까요?'), ('human', '내이름이머라고 ?.'), ('ai', 'Teddy님이라고 하셨습니다! 맞나요?')]
[('ai', '안녕하세요, Teddy님! 만나서 반갑습니다. 어떻게 도와드릴까요?'), ('human', '내이름이머라고 ?.'), ('ai', 'Teddy님이라고 하셨습니다! 맞나요?')]


In [19]:
question="맞아 내직업은 개발자야 잘알아둬 ?."

for event in graph.stream({"messages": [question]}, config=config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('ai', '안녕하세요, Teddy님! 만나서 반갑습니다. 어떻게 도와드릴까요?'), ('human', '내이름이머라고 ?.'), ('ai', 'Teddy님이라고 하셨습니다! 맞나요?'), ('human', '맞아 내직업은 개발자야 잘알아둬 ?.')]
[('ai', '안녕하세요, Teddy님! 만나서 반갑습니다. 어떻게 도와드릴까요?'), ('human', '내이름이머라고 ?.'), ('ai', 'Teddy님이라고 하셨습니다! 맞나요?'), ('human', '맞아 내직업은 개발자야 잘알아둬 ?.'), ('ai', '네, Teddy님은 개발자시군요! 개발 관련 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.')]
[('ai', 'Teddy님이라고 하셨습니다! 맞나요?'), ('human', '맞아 내직업은 개발자야 잘알아둬 ?.'), ('ai', '네, Teddy님은 개발자시군요! 개발 관련 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.')]


In [23]:
question="내이름이 머라고 ?"

for event in graph.stream({"messages": [question]}, config=config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('ai', '미술학을 전공하셨군요! 정말 흥미로운 전공입니다. 미술과 개발을 결합한 프로젝트나 관심 있는 분야가 있으신가요? 예를 들어, 디지털 아트, 인터랙티브 아트, 또는 게임 디자인 같은 분야에 관심이 있으신지 궁금합니다.'), ('human', '아니 딱히 없는데?'), ('ai', '그렇군요! 미술학을 전공하셨지만 특별한 분야에 관심이 없으시다면, 다른 취미나 관심사에 대해 이야기해 보실까요? 예를 들어, 좋아하는 예술가나 작품, 혹은 최근에 감명 깊게 본 전시회 같은 것이 있을까요?'), ('human', '내이름이 머라고 ?')]
[('ai', '미술학을 전공하셨군요! 정말 흥미로운 전공입니다. 미술과 개발을 결합한 프로젝트나 관심 있는 분야가 있으신가요? 예를 들어, 디지털 아트, 인터랙티브 아트, 또는 게임 디자인 같은 분야에 관심이 있으신지 궁금합니다.'), ('human', '아니 딱히 없는데?'), ('ai', '그렇군요! 미술학을 전공하셨지만 특별한 분야에 관심이 없으시다면, 다른 취미나 관심사에 대해 이야기해 보실까요? 예를 들어, 좋아하는 예술가나 작품, 혹은 최근에 감명 깊게 본 전시회 같은 것이 있을까요?'), ('human', '내이름이 머라고 ?'), ('ai', '죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!')]
[('ai', '그렇군요! 미술학을 전공하셨지만 특별한 분야에 관심이 없으시다면, 다른 취미나 관심사에 대해 이야기해 보실까요? 예를 들어, 좋아하는 예술가나 작품, 혹은 최근에 감명 깊게 본 전시회 같은 것이 있을까요?'), ('human', '내이름이 머라고 ?'), ('ai', '죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!')]
